In [1]:
from __future__ import print_function
import sklearn
import sklearn.datasets

import sklearn.ensemble
import numpy as np
import lime


import lime.lime_tabular
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
import pandas as pd

# np.random.seed(1)

In [2]:
feature_names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]
feature_names_ = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country", "Target"]


In [3]:
data = np.genfromtxt('adult.data', delimiter=', ', dtype=str)
data

array([['39', 'State-gov', '77516', ..., '40', 'United-States', '<=50K'],
       ['50', 'Self-emp-not-inc', '83311', ..., '13', 'United-States',
        '<=50K'],
       ['38', 'Private', '215646', ..., '40', 'United-States', '<=50K'],
       ...,
       ['58', 'Private', '151910', ..., '40', 'United-States', '<=50K'],
       ['22', 'Private', '201490', ..., '20', 'United-States', '<=50K'],
       ['52', 'Self-emp-inc', '287927', ..., '40', 'United-States',
        '>50K']], dtype='<U26')

In [4]:
df = pd.DataFrame(data=data,  columns=feature_names_)

In [5]:
labels = data[:,14]
print(labels)
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
print(labels)
class_names = le.classes_
print(class_names)
data = data[:,:-1]


['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '>50K']
[0 0 0 ... 0 0 1]
['<=50K' '>50K']


In [6]:
categorical_features = [1,3,5, 6,7,8,9,13]
m = 1

In [7]:
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    print(data[:, feature])
    categorical_names[feature] = le.classes_
    print(categorical_names[feature])

['7' '6' '4' ... '4' '4' '5']
['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
['9' '9' '11' ... '11' '11' '11']
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
['4' '2' '0' ... '6' '4' '2']
['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed']
['1' '4' '6' ... '1' '1' '4']
['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
['1' '0' '1' ... '4' '3' '5']
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
['4' '4' '4' ... '4' '4' '4']
['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White']
['1' '1' '1' ... '0' '1' 

In [8]:
data = data.astype(float)
print(data)


[[3.90000e+01 7.00000e+00 7.75160e+04 ... 0.00000e+00 4.00000e+01
  3.90000e+01]
 [5.00000e+01 6.00000e+00 8.33110e+04 ... 0.00000e+00 1.30000e+01
  3.90000e+01]
 [3.80000e+01 4.00000e+00 2.15646e+05 ... 0.00000e+00 4.00000e+01
  3.90000e+01]
 ...
 [5.80000e+01 4.00000e+00 1.51910e+05 ... 0.00000e+00 4.00000e+01
  3.90000e+01]
 [2.20000e+01 4.00000e+00 2.01490e+05 ... 0.00000e+00 2.00000e+01
  3.90000e+01]
 [5.20000e+01 5.00000e+00 2.87927e+05 ... 0.00000e+00 4.00000e+01
  3.90000e+01]]


In [9]:
df = pd.DataFrame(data=data,  columns=feature_names)
df.corr()


,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country
Age,1.000000,0.003787,-0.076646,-0.010508,0.036527,-0.266288,-0.020947,-0.263698,0.028718,0.088832,0.077674,0.057775,0.068756,-0.001151
Workclass,0.003787,1.000000,-0.016656,0.023513,0.052085,-0.064731,0.254892,-0.090461,0.049742,0.095981,0.033835,0.012216,0.138962,-0.007690
fnlwgt,-0.076646,-0.016656,1.000000,-0.028145,-0.043195,0.028153,0.001597,0.008931,-0.021291,0.026858,0.000432,-0.010252,-0.018768,-0.051966
Education,-0.010508,0.023513,-0.028145,1.000000,0.359153,-0.038407,-0.021260,-0.010876,0.014131,-0.027356,0.030046,0.016746,0.055510,0.064288
Education-Num,0.036527,0.052085,-0.043195,0.359153,1.000000,-0.069304,0.109697,-0.094153,0.031838,0.012280,0.122630,0.079923,0.148123,0.050840
Marital Status,-0.266288,-0.064731,0.028153,-0.038407,-0.069304,1.000000,-0.009654,0.185451,-0.068013,-0.129314,-0.043393,-0.034187,-0.190519,-0.023819
Occupation,-0.020947,0.254892,0.001597,-0.021260,0.109697,-0.009654,1.000000,-0.075607,0.006763,0.080296,0.025505,0.017987,0.080383,-0.012543
Relationship,-0.263698,-0.090461,0.008931,-0.010876,-0.094153,0.185451,-0.075607,1.000000,-0.116055,-0.582454,-0.057919,-0.061062,-0.248974,-0.005507
Race,0.028718,0.049742,-0.021291,0.014131,0.031838,-0.068013,0.006763,-0.116055,1.000000,0.087204,0.011145,0.018899,0.041910,0.137852
Sex,0.088832,0.095981,0.026858,-0.027356,0.012280,-0.129314,0.080296,-0.582454,0.087204,1.000000,0.048480,0.045567,0.229309,-0.008119


In [10]:
from sklearn.compose import ColumnTransformer

encoder = ColumnTransformer(
    [('one_hot_encoder', sklearn.preprocessing.OneHotEncoder(categories='auto'),  [1,3,5, 6,7,8,9,13])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)


In [11]:
np.random.seed(m)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, test_size= 0.2)

In [12]:
from imblearn.over_sampling import SMOTE
unique, counts = np.unique(labels_train, return_counts=True)
print(dict(zip(unique, counts)))
sm = SMOTE(ratio='auto')
train_res, labels_train_res = sm.fit_sample(train, labels_train)
unique, counts = np.unique(labels_train_res, return_counts=True)
print(dict(zip(unique, counts)))

{0: 19694, 1: 6354}
{0: 19694, 1: 19694}


In [13]:
data = []
data = np.concatenate([train_res,test])
data.shape


(45901, 14)

In [14]:
encoder.fit(data)
encoded_train = encoder.transform(train_res)
encoded_test = encoder.transform(test)

In [15]:
from sklearn.linear_model import LogisticRegression
gbtree =  LogisticRegression()
gbtree.fit(encoded_train, labels_train_res)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')


In [17]:
data[1]

array([5.4000e+01, 6.0000e+00, 1.0501e+05, 9.0000e+00, 1.3000e+01,
       2.0000e+00, 4.0000e+00, 0.0000e+00, 4.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 4.0000e+01, 3.9000e+01])

In [18]:
from pylab import *
yhat = gbtree.predict_proba(encoded_test)
probs = yhat[:, 1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [sklearn.metrics.f1_score(labels_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))


Threshold=0.401, F-Score=0.66528


In [19]:
thresholds[ix]

0.401

In [20]:
sklearn.metrics.accuracy_score(labels_test,to_labels(probs, thresholds[ix]))


0.8265008444649163

In [21]:
from sklearn.metrics import confusion_matrix

In [22]:
predict_fn = lambda x: gbtree.predict_proba(encoder.transform(x)).astype(float)


In [23]:
print("\nAccuracy is ", sklearn.metrics.accuracy_score(labels_test, to_labels(probs, thresholds[ix]))*100)
print("\n\nPrecision Value is\n", sklearn.metrics.classification_report(labels_test, to_labels(probs, thresholds[ix])))



Accuracy is  82.65008444649163


Precision Value is
               precision    recall  f1-score   support

           0       0.92      0.85      0.88      5026
           1       0.59      0.76      0.67      1487

   micro avg       0.83      0.83      0.83      6513
   macro avg       0.76      0.80      0.77      6513
weighted avg       0.85      0.83      0.83      6513



In [24]:
sklearn.metrics.roc_curve( labels_test, to_labels(probs, thresholds[ix]))

(array([0.        , 0.15240748, 1.        ]),
 array([0.        , 0.75521184, 1.        ]),
 array([2, 1, 0]))

### Another model trained after removing column 'Sex'

In [25]:
feature_names1 = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation",
                  "Relationship", "Race", "Capital Gain", "Capital Loss","Hours per week", "Country"]
data = np.genfromtxt('adult.data', delimiter=', ', dtype=str)

In [26]:
data = np.delete(data, 9, axis = 1)


In [27]:
labels = data[:,13]
print(labels)
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
print(labels)
class_names = le.classes_
print(class_names)
data = data[:,:-1]
data.shape


['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '>50K']
[0 0 0 ... 0 0 1]
['<=50K' '>50K']


(32561, 13)

In [28]:
categorical_features = [1,3,5, 6,7,8,12]
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    print(data[:, feature])
    categorical_names[feature] = le.classes_
    print(categorical_names[feature])
data = data.astype(float)

['7' '6' '4' ... '4' '4' '5']
['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
['9' '9' '11' ... '11' '11' '11']
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
['4' '2' '0' ... '6' '4' '2']
['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed']
['1' '4' '6' ... '1' '1' '4']
['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
['1' '0' '1' ... '4' '3' '5']
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
['4' '4' '4' ... '4' '4' '4']
['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White']
['39' '39' '39' ... '39' 

In [29]:
from sklearn.compose import ColumnTransformer

encoder1 = ColumnTransformer(
    [('one_hot_encoder', sklearn.preprocessing.OneHotEncoder(categories='auto'),  [1,3,5, 6,7,8,12])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
encoder1.fit(data)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('one_hot_encoder', OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True), [1, 3, 5, 6, 7, 8, 12])])

In [30]:
np.random.seed(m)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, test_size= 0.2)

In [31]:
from imblearn.over_sampling import SMOTE
unique, counts = np.unique(labels_train, return_counts=True)
print(dict(zip(unique, counts)))
sm = SMOTE(ratio='auto')
train_res, labels_train_res = sm.fit_sample(train, labels_train)
unique, counts = np.unique(labels_train_res, return_counts=True)
print(dict(zip(unique, counts)))

{0: 19694, 1: 6354}
{0: 19694, 1: 19694}


In [32]:
data = []
data = np.concatenate([train_res,test])
data.shape


(45901, 13)

In [33]:
encoder1 = encoder1.fit(data)
encoded_train = encoder1.transform(train_res)
encoded_test = encoder1.transform(test)
d1 = encoder1.transform(data)

In [34]:

gbtree1 = LogisticRegression()
gbtree1.fit(encoded_train, labels_train_res)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [35]:
from pylab import *
yhat = gbtree1.predict_proba(encoded_test)
probs = yhat[:, 1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [sklearn.metrics.f1_score(labels_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.378, F-Score=0.67009


In [36]:
thresholds[ix]

0.378

In [37]:
sklearn.metrics.accuracy_score(labels_test,to_labels(probs, thresholds[ix] ) )


0.8324888684170121

In [38]:
gbtree1.predict_proba(d1[20])

array([[0.90784903, 0.09215097]])

In [39]:
def predict_fn1(x):
    x = np.delete(x, 9, axis = 1)
    return gbtree1.predict_proba(encoder1.transform(x)).astype(float)

In [40]:
from sklearn.metrics import confusion_matrix
result1= confusion_matrix(labels_test, to_labels(probs,thresholds[ix]))
result1

array([[4314,  712],
       [ 379, 1108]])

In [41]:
print("\nAccuracy is ", sklearn.metrics.accuracy_score(labels_test, to_labels(probs, thresholds[ix]))*100)
print("\n\nPrecision Value is\n", sklearn.metrics.classification_report(labels_test, to_labels(probs, thresholds[ix])))


Accuracy is  83.24888684170121


Precision Value is
               precision    recall  f1-score   support

           0       0.92      0.86      0.89      5026
           1       0.61      0.75      0.67      1487

   micro avg       0.83      0.83      0.83      6513
   macro avg       0.76      0.80      0.78      6513
weighted avg       0.85      0.83      0.84      6513



### Another model trained after removing column 'Race'


In [42]:
feature_names2 = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", "Relationship","Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]
data = np.genfromtxt('adult.data', delimiter=', ', dtype=str)

In [43]:
data = np.delete(data, 8, axis = 1)

In [44]:
labels = data[:,13]
print(labels)
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
print(labels)
class_names = le.classes_
print(class_names)
data = data[:,:-1]
data.shape

['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '>50K']
[0 0 0 ... 0 0 1]
['<=50K' '>50K']


(32561, 13)

In [45]:
categorical_features = [1,3,5, 6,7,8,12]
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    print(data[:, feature])
    categorical_names[feature] = le.classes_
    print(categorical_names[feature])
data = data.astype(float)


['7' '6' '4' ... '4' '4' '5']
['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
['9' '9' '11' ... '11' '11' '11']
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
['4' '2' '0' ... '6' '4' '2']
['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed']
['1' '4' '6' ... '1' '1' '4']
['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
['1' '0' '1' ... '4' '3' '5']
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
['1' '1' '1' ... '0' '1' '0']
['Female' 'Male']
['39' '39' '39' ... '39' '39' '39']
['?' 'Cambodia' 'Canada' 'China' 'Colum

In [46]:
data = data.astype(float)

In [47]:
from sklearn.compose import ColumnTransformer

encoder2 = ColumnTransformer(
    [('one_hot_encoder', sklearn.preprocessing.OneHotEncoder(categories='auto'),  [1,3,5, 6,7,8,12])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
encoder2.fit(data)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('one_hot_encoder', OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True), [1, 3, 5, 6, 7, 8, 12])])

In [48]:
np.random.seed(m)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, test_size= 0.2)

In [49]:
from imblearn.over_sampling import SMOTE
unique, counts = np.unique(labels_train, return_counts=True)
print(dict(zip(unique, counts)))
sm = SMOTE(ratio='auto')
train_res, labels_train_res = sm.fit_sample(train, labels_train)
unique, counts = np.unique(labels_train_res, return_counts=True)
print(dict(zip(unique, counts)))

{0: 19694, 1: 6354}
{0: 19694, 1: 19694}


In [50]:
data = []
data = np.concatenate([train_res,test])
data.shape


(45901, 13)

In [51]:
encoder2 = encoder2.fit(data)
encoded_train = encoder2.transform(train_res)
encoded_test = encoder2.transform(test)
d2 = encoder2.transform(data)

In [52]:
gbtree2 = LogisticRegression()
gbtree2.fit(encoded_train, labels_train_res)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [53]:
from pylab import *
yhat = gbtree2.predict_proba(encoded_test)
probs = yhat[:, 1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [sklearn.metrics.f1_score(labels_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.440, F-Score=0.66811


In [54]:
thresholds[ix]

0.44

In [55]:
sklearn.metrics.accuracy_score(labels_test,to_labels(probs,thresholds[ix]) )

0.8354061108552127

In [56]:
gbtree2.predict_proba(d2[20])

array([[0.89001596, 0.10998404]])

In [57]:
def predict_fn2(x):
    x = np.delete(x, 8, axis = 1)
    return gbtree2.predict_proba(encoder2.transform(x)).astype(float)

In [58]:
from sklearn.metrics import confusion_matrix
result2= confusion_matrix(labels_test, to_labels(probs, 0.301))
result2

array([[3923, 1103],
       [ 233, 1254]])

In [59]:
print("\nAccuracy is ", sklearn.metrics.accuracy_score(labels_test, to_labels(probs, thresholds[ix]))*100)
print("\n\nPrecision Value is\n", sklearn.metrics.classification_report(labels_test, to_labels(probs, thresholds[ix])))


Accuracy is  83.54061108552126


Precision Value is
               precision    recall  f1-score   support

           0       0.91      0.87      0.89      5026
           1       0.62      0.73      0.67      1487

   micro avg       0.84      0.84      0.84      6513
   macro avg       0.77      0.80      0.78      6513
weighted avg       0.85      0.84      0.84      6513



### Another model after removing Marital Status

In [60]:
feature_names4 = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num","Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]
data = np.genfromtxt('adult.data', delimiter=', ', dtype=str)

In [61]:
data = np.delete(data, 5, axis = 1)

In [62]:
labels = data[:,13]
print(labels)
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
print(labels)
class_names = le.classes_
print(class_names)
data = data[:,:-1]
data.shape

['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '>50K']
[0 0 0 ... 0 0 1]
['<=50K' '>50K']


(32561, 13)

In [63]:
categorical_features = [1,3,5, 6,7,8,12]
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    print(data[:, feature])
    categorical_names[feature] = le.classes_
    print(categorical_names[feature])
data = data.astype(float)

['7' '6' '4' ... '4' '4' '5']
['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
['9' '9' '11' ... '11' '11' '11']
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
['1' '4' '6' ... '1' '1' '4']
['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
['1' '0' '1' ... '4' '3' '5']
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
['4' '4' '4' ... '4' '4' '4']
['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White']
['1' '1' '1' ... '0' '1' '0']
['Female' 'Male']
['39' '39' '39' ... '39' '39' '39']
['?' 'Cambodia' 'Canada' 'China' 'Columbia' 'Cuba' 'Dominican-Republic'
 'Ecuador' 'El-S

In [64]:
from sklearn.compose import ColumnTransformer

encoder4 = ColumnTransformer(
    [('one_hot_encoder', sklearn.preprocessing.OneHotEncoder(categories='auto'),  [1,3,5, 6,7,8,12])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
encoder4.fit(data)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('one_hot_encoder', OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True), [1, 3, 5, 6, 7, 8, 12])])

In [65]:
np.random.seed(m)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, test_size= 0.20)

In [66]:
from imblearn.over_sampling import SMOTE
unique, counts = np.unique(labels_train, return_counts=True)
print(dict(zip(unique, counts)))
sm = SMOTE(ratio='auto')
train_res, labels_train_res = sm.fit_sample(train, labels_train)
unique, counts = np.unique(labels_train_res, return_counts=True)
print(dict(zip(unique, counts)))

{0: 19694, 1: 6354}
{0: 19694, 1: 19694}


In [67]:
data = []
data = np.concatenate([train_res,test])
data.shape


(45901, 13)

In [68]:
encoder4 = encoder4.fit(data)
encoded_train = encoder4.transform(train_res)
encoded_test = encoder4.transform(test)
d4 = encoder4.transform(data)

In [69]:
gbtree4 = LogisticRegression()
gbtree4.fit(encoded_train, labels_train_res)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [70]:
from pylab import *
yhat = gbtree4.predict_proba(encoded_test)
probs = yhat[:, 1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [sklearn.metrics.f1_score(labels_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.426, F-Score=0.66980


In [71]:
thresholds[ix]

0.426

In [72]:
sklearn.metrics.accuracy_score(labels_test, to_labels(probs, thresholds[ix]))


0.8380162751420236

In [73]:
gbtree4.predict_proba(d4[20])
def predict_fn4(x):
    x = np.delete(x,5,axis =1)
    return gbtree4.predict_proba(encoder4.transform(x)).astype(float)

In [74]:
from sklearn.metrics import confusion_matrix
result4= confusion_matrix(labels_test, to_labels(probs, thresholds[ix]))
result4

array([[4388,  638],
       [ 417, 1070]])

In [75]:

print("\nAccuracy is ", sklearn.metrics.accuracy_score(labels_test, to_labels(probs, thresholds[ix]))*100)
print("\n\nPrecision Value is\n", sklearn.metrics.classification_report(labels_test, to_labels(probs,thresholds[ix])))


Accuracy is  83.80162751420237


Precision Value is
               precision    recall  f1-score   support

           0       0.91      0.87      0.89      5026
           1       0.63      0.72      0.67      1487

   micro avg       0.84      0.84      0.84      6513
   macro avg       0.77      0.80      0.78      6513
weighted avg       0.85      0.84      0.84      6513



### Another model trained after removing column 'Race' and 'Sex' and 'Marital Status'


In [76]:
feature_names3 = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num","Occupation", "Relationship", "Capital Gain", "Capital Loss","Hours per week", "Country"]
data = np.genfromtxt('adult.data', delimiter=', ', dtype=str)

In [77]:
data = np.delete(data, 8, axis = 1)
data = np.delete(data, 8, axis = 1)
data = np.delete(data, 5, axis = 1)

In [78]:
labels = data[:,11]
print(labels)
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
print(labels)
class_names = le.classes_
print(class_names)
data = data[:,:-1]
data.shape

['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '>50K']
[0 0 0 ... 0 0 1]
['<=50K' '>50K']


(32561, 11)

In [79]:
categorical_features = [1,3,5, 6,10]
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    print(data[:, feature])
    categorical_names[feature] = le.classes_
    print(categorical_names[feature])


['7' '6' '4' ... '4' '4' '5']
['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
['9' '9' '11' ... '11' '11' '11']
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
['1' '4' '6' ... '1' '1' '4']
['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
['1' '0' '1' ... '4' '3' '5']
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
['39' '39' '39' ... '39' '39' '39']
['?' 'Cambodia' 'Canada' 'China' 'Columbia' 'Cuba' 'Dominican-Republic'
 'Ecuador' 'El-Salvador' 'England' 'France' 'Germany' 'Greece' 'Guatemala'
 'Haiti' 'Holand-Netherlands' 'Honduras' 'Hong' 'Hungary' 'India' 'Iran'
 'Ireland' 'It

In [80]:
data = data.astype(float)
# d_male = d_male.astype(float)
# d_female = d_female.astype(float)

In [81]:
from sklearn.compose import ColumnTransformer

encoder3 = ColumnTransformer(
    [('one_hot_encoder', sklearn.preprocessing.OneHotEncoder(categories='auto'),  [1,3,5, 6,10])],   # The column numbers to be transformed (here is [0] but can be [0, 1, 3])
    remainder='passthrough'                                         # Leave the rest of the columns untouched
)
encoder3.fit(data)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('one_hot_encoder', OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True), [1, 3, 5, 6, 10])])

In [82]:
np.random.seed(m)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, test_size= 0.20)

In [83]:
from imblearn.over_sampling import SMOTE
unique, counts = np.unique(labels_train, return_counts=True)
print(dict(zip(unique, counts)))
sm = SMOTE(ratio='auto')
train_res, labels_train_res = sm.fit_sample(train, labels_train)
unique, counts = np.unique(labels_train_res, return_counts=True)
print(dict(zip(unique, counts)))

{0: 19694, 1: 6354}
{0: 19694, 1: 19694}


In [84]:
data = []
data = np.concatenate([train_res,test])
data.shape

(45901, 11)

In [85]:
encoder3 = encoder3.fit(data)
encoded_train = encoder3.transform(train_res)
encoded_test = encoder3.transform(test)
d3 = encoder3.transform(data)

In [86]:
gbtree3 =LogisticRegression()
gbtree3.fit(encoded_train, labels_train_res)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [87]:
from pylab import *
yhat = gbtree3.predict_proba(encoded_test)
probs = yhat[:, 1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [sklearn.metrics.f1_score(labels_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.364, F-Score=0.66400


In [88]:
sklearn.metrics.accuracy_score(labels_test, to_labels(probs, thresholds[ix]))


0.8197451251343467

In [89]:
gbtree3.predict_proba(d3[20])
def predict_fn3(x):
    x = np.delete(x, 8, axis = 1)
    x = np.delete(x,8,axis =1)
    x = np.delete(x,5,axis =1)
    return gbtree3.predict_proba(encoder3.transform(x)).astype(float)

In [90]:
thresholds[ix]


0.364

In [91]:
print("\nAccuracy is ", sklearn.metrics.accuracy_score(labels_test, to_labels(probs, thresholds[ix]))*100)
print("\n\nPrecision Value is\n", sklearn.metrics.classification_report(labels_test, to_labels(probs, thresholds[ix])))


Accuracy is  81.97451251343468


Precision Value is
               precision    recall  f1-score   support

           0       0.93      0.83      0.88      5026
           1       0.58      0.78      0.66      1487

   micro avg       0.82      0.82      0.82      6513
   macro avg       0.75      0.81      0.77      6513
weighted avg       0.85      0.82      0.83      6513



### Ensemble of 4 models

In [92]:
feature_names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]
data = np.genfromtxt('adult.data', delimiter=', ', dtype=str)


In [93]:
labels = data[:,14]
print(labels)
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
print(labels)
class_names = le.classes_
print(class_names)
data = data[:,:-1]
data.shape

['<=50K' '<=50K' '<=50K' ... '<=50K' '<=50K' '>50K']
[0 0 0 ... 0 0 1]
['<=50K' '>50K']


(32561, 14)

In [94]:
categorical_features = [1,3,5, 6,7,8,9,13]

In [95]:
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    print(data[:, feature])
    categorical_names[feature] = le.classes_
    print(categorical_names[feature])

['7' '6' '4' ... '4' '4' '5']
['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
['9' '9' '11' ... '11' '11' '11']
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
['4' '2' '0' ... '6' '4' '2']
['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed']
['1' '4' '6' ... '1' '1' '4']
['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
['1' '0' '1' ... '4' '3' '5']
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
['4' '4' '4' ... '4' '4' '4']
['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White']
['1' '1' '1' ... '0' '1' 

In [96]:
data = data.astype(float)
# d_male = d_male.astype(float)
# d_female = d_female.astype(float)
print(data)

[[3.90000e+01 7.00000e+00 7.75160e+04 ... 0.00000e+00 4.00000e+01
  3.90000e+01]
 [5.00000e+01 6.00000e+00 8.33110e+04 ... 0.00000e+00 1.30000e+01
  3.90000e+01]
 [3.80000e+01 4.00000e+00 2.15646e+05 ... 0.00000e+00 4.00000e+01
  3.90000e+01]
 ...
 [5.80000e+01 4.00000e+00 1.51910e+05 ... 0.00000e+00 4.00000e+01
  3.90000e+01]
 [2.20000e+01 4.00000e+00 2.01490e+05 ... 0.00000e+00 2.00000e+01
  3.90000e+01]
 [5.20000e+01 5.00000e+00 2.87927e+05 ... 0.00000e+00 4.00000e+01
  3.90000e+01]]


In [97]:
np.random.seed(m)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, test_size= 0.20)

In [98]:
type(test)

numpy.ndarray

In [99]:
explainer = lime.lime_tabular.LimeTabularExplainer(train ,feature_names = feature_names,class_names=class_names,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3)

Averaging the predictions to get the final model

In [100]:
def predict_fne(x):
    t = predict_fn1(x)+ predict_fn2(x) + predict_fn3(x) + predict_fn4(x)
    return t/4

In [101]:
def predicte(test):
    arr = []
    for i in range(len(test)):
        t = test[i].reshape(1,-1)
        val = predict_fne(t)
        if(val[0][1] >= 0.5):
            arr = np.append(arr, val[0][1])
        else:
            arr = np.append(arr, val[0][1])
    return arr

In [102]:
t = predicte(test)

In [103]:
from pylab import *
probs = t
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [sklearn.metrics.f1_score(labels_test, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.450, F-Score=0.66838


In [104]:
np.random.seed(m)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80, test_size= 0.20)

In [105]:
explainer = lime.lime_tabular.LimeTabularExplainer(train ,feature_names = feature_names,class_names=class_names,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3)

In [106]:
expe = explainer.explain_instance(test[1], predict_fne, num_features=10)
expe.show_in_notebook(show_all=False)

[(10, -0.6079761202194703), (4, -0.16029739629827627), (11, -0.12504239647811086), (12, -0.09480168290666545), (5, -0.09088337442059843), (6, -0.07411391080238071), (0, -0.057287364445373724), (9, 0.052275128026544396), (13, 0.030165299080121083), (1, 0.02706895227344765)]
[1]


In [107]:
from sklearn.metrics import confusion_matrix
resulte =  confusion_matrix(labels_test, to_labels(probs, thresholds[ix]))
resulte

array([[4445,  581],
       [ 449, 1038]])

In [108]:
print("\nAccuracy is ", sklearn.metrics.accuracy_score(labels_test, to_labels(probs,thresholds[ix]))*100)
print("\n\nPrecision Value is\n", sklearn.metrics.classification_report(labels_test, to_labels(probs,thresholds[ix])))


Accuracy is  84.18547520343928


Precision Value is
               precision    recall  f1-score   support

           0       0.91      0.88      0.90      5026
           1       0.64      0.70      0.67      1487

   micro avg       0.84      0.84      0.84      6513
   macro avg       0.77      0.79      0.78      6513
weighted avg       0.85      0.84      0.84      6513



Used Submodular Pick to get the instances, then used these instances to get the global explanations for original and ensemble model respectively

In [109]:
from lime import submodular_pick
sp_obj = submodular_pick.SubmodularPick(explainer, train, predict_fn, sample_size=500, num_features=10, num_exps_desired=50)
type(sp_obj)

[(10, -0.6134171442023233), (5, -0.1843257395509945), (4, 0.1816893958359554), (11, -0.12621166366876924), (9, -0.09734275327839131), (12, -0.0898452808972982), (8, 0.05759430770912324), (13, 0.05113891548367209), (0, -0.04658881987367523), (3, -0.037648580526424384)]
[(10, -0.6349370520155574), (11, -0.13052962866432546), (9, -0.10005503297343882), (12, -0.0926772738166884), (5, -0.08903188749620829), (6, 0.06867468016067134), (8, 0.0512143382038908), (2, 0.036774126888019246), (7, -0.02914364917533042), (13, 0.028484363158915647)]
[(10, -0.6018854629806721), (5, 0.200153206090381), (11, -0.1416500558570153), (9, 0.1086029197158133), (8, 0.059698704075780554), (0, -0.055956469029576024), (13, 0.04218490645440397), (7, 0.02997785104951727), (1, 0.029233469536137768), (12, 0.026506789163229873)]
[(10, -0.6197836084840321), (5, -0.17323358951414128), (11, -0.1429231672310443), (9, -0.09049210236610171), (12, -0.08893176748872106), (4, 0.06744013787788629), (8, 0.04888351393871579), (13, 

[(10, -0.631720867069393), (5, 0.19619919772336664), (11, -0.12664816844953686), (9, 0.09444286973063618), (12, -0.0827839169894505), (0, -0.0511393968432653), (8, 0.046431496299884537), (13, 0.035876696831206624), (2, 0.02643257879092996), (6, 0.025049512496599077)]
[(10, -0.6294110364847512), (5, 0.20104267747773888), (4, 0.1983822392005034), (11, -0.1366994254619311), (9, 0.10448366375887337), (12, 0.10111728229507298), (13, 0.05825899473335925), (8, 0.05488120319222445), (7, 0.03310777459350388), (3, -0.02783802448282988)]
[(10, 0.5999657014952502), (5, 0.2024038269170879), (4, -0.16095145933436655), (11, -0.1395613634407627), (9, 0.10287793527234443), (12, -0.08445272844470167), (0, 0.07899786635814643), (6, 0.07181690357723818), (8, 0.047132408014679605), (7, 0.031654535986703584)]
[(10, -0.6123554349115937), (5, 0.20746264301867035), (4, 0.19879703943928725), (11, -0.12832867369129636), (9, 0.10122135709407501), (12, 0.0922985633924318), (0, 0.07903502861433283), (1, 0.060960022

[(10, -0.6210252211367627), (5, 0.19479052406197797), (4, -0.15666234578011162), (11, -0.14684863011976163), (9, 0.10205508955061127), (0, 0.06728300094563805), (6, 0.06566082360534625), (1, 0.06084062121579185), (8, 0.04944029694882062), (13, 0.04664843778475311)]
[(10, -0.6439228274105796), (11, -0.15641520660179098), (9, -0.09340792116681608), (12, -0.08735856984063577), (5, -0.0807794497584058), (8, -0.05671158880913031), (6, -0.05484580605969113), (7, -0.04443083259763181), (13, 0.028509912880353597), (3, -0.025805254877984084)]
[(10, -0.6144691756108625), (5, -0.17477236669144308), (4, -0.15264750106539876), (11, -0.1297340185331648), (12, 0.10511376548139126), (9, 0.10088614033894863), (7, -0.05462356202962159), (0, -0.04774337691769718), (8, 0.03206359762583229), (13, 0.025648640009256592)]
[(10, -0.6124411814141602), (5, -0.17870526974570794), (4, -0.14429913411061857), (11, -0.13154144965204922), (12, -0.09546214262047426), (9, -0.09347158953572828), (7, -0.06491237519096772)

[(10, 0.6206364539006931), (4, -0.15489230270912094), (11, -0.11376868892244875), (9, 0.08784481631601061), (5, -0.08358983977216325), (8, 0.05006742317069646), (13, 0.03213816836986005), (7, -0.030012414154431843), (12, 0.02706804528899675), (1, 0.022748822695680952)]
[(10, -0.6134076285989991), (4, 0.18710773477804513), (5, -0.17574502540568204), (11, -0.138166633795569), (12, -0.09222941015082183), (9, 0.09156885458457081), (0, -0.06326034342599954), (7, -0.061027918334469695), (6, -0.03981365639140358), (13, 0.03734712411659313)]
[(10, -0.6239837378778783), (5, -0.1835938386599745), (11, -0.13278626145526048), (9, 0.09731969851563098), (12, -0.08282478261394019), (6, 0.07859999881414588), (13, -0.0724203505269844), (7, -0.060511549254552224), (3, -0.025318061089026), (0, -0.022025475083294013)]
[(10, -0.6310229150875986), (5, 0.2010298079897361), (4, 0.1999480570527953), (11, -0.1330122281020029), (9, 0.10430299861604696), (12, -0.08945664090035876), (0, 0.06301507652808293), (8, 0

[(10, -0.6247040422648169), (5, 0.19925675725189307), (11, 0.14088450755840465), (9, 0.10940521407906202), (12, 0.0963933640873868), (4, 0.05350141344422097), (8, 0.04780552855571479), (13, 0.042284954609423005), (7, 0.03176071112482169), (1, 0.021747837205725866)]
[(10, -0.6132619224008551), (4, 0.18821208235933806), (5, -0.17276989547934032), (11, -0.1341338905965414), (9, -0.09465073680798632), (12, -0.08345220475994196), (6, 0.07818070493587399), (3, 0.07350724138261276), (8, 0.04936355461422297), (13, -0.035217780428387906)]
[(10, 0.6172828113301767), (5, 0.2007660126522231), (11, -0.17084984061002204), (9, 0.09505505683286576), (6, 0.09260219581999955), (12, -0.08851633318188916), (0, 0.06344951483127437), (8, 0.0456678405551589), (1, -0.03474093953316336), (13, 0.034549356135004906)]
[(10, -0.5993220071294361), (5, 0.19884745097276588), (4, -0.15059159883921477), (11, -0.1393260292656347), (9, 0.1006261091944582), (12, -0.08688648557676217), (0, 0.079642700501209), (3, 0.0549277

...

[(10, -0.6266065999577849), (4, 0.18944908250574127), (11, -0.14926738325898614), (9, 0.09992219542794573), (12, -0.09646672659205101), (5, -0.09355243464712817), (8, 0.04733591961353361), (3, 0.043678212718946945), (7, -0.026740711791898724), (1, -0.024456182979989347)]
[(10, -0.6376816663303068), (5, 0.20005160905322963), (11, -0.1512282926843155), (12, -0.10767851507657811), (9, 0.09446209483489483), (0, 0.07314404348183755), (6, 0.06906329128769403), (8, 0.05513720772337363), (4, 0.05013294418430114), (13, 0.03740178518129699)]
[(10, -0.6283598194592376), (4, 0.19255202212644854), (5, -0.1672874785184295), (11, -0.10951128102527202), (9, -0.10036275331532912), (12, -0.0919701800433475), (0, -0.045648578855067005), (13, 0.04473046644185806), (8, 0.043071208065515836), (3, -0.04149580962404092)]
[(10, -0.6571405061001313), (5, -0.17631245790604783), (4, -0.14883819262863862), (11, -0.1284304720979018), (9, -0.09287139243066096), (12, -0.08161390675645112), (8, 0.0488832752462413), (1

[(10, -0.623283245286485), (5, -0.17507555095037233), (4, -0.14557947113454608), (11, -0.11660513128425296), (9, -0.09867452314765346), (12, -0.0899705487733458), (0, -0.05082730802263654), (7, -0.047190456168414746), (8, 0.04648135888966609), (13, 0.03710337407557096)]
[(10, -0.6278156050010545), (5, -0.17348055583493702), (4, -0.15344353682271994), (11, -0.11755752842211417), (9, 0.09797702019888423), (12, -0.08474314854814916), (3, 0.07747725185140798), (0, -0.05540367265213703), (13, -0.04712884605170497), (8, 0.035996547278038936)]
[(10, -0.6172841420968148), (5, 0.1901230651163696), (4, -0.15902112967976806), (11, -0.13968215447826623), (12, 0.09660965262149328), (9, 0.09157487200546406), (3, 0.07251309371122361), (13, -0.06621784854545452), (8, 0.052997291378548146), (6, -0.025013253427527866)]
[(10, -0.6213836576673855), (5, 0.1979057446452129), (4, 0.182270839011056), (11, -0.133010498627174), (12, 0.1026949501029229), (9, 0.10146168033088751), (1, -0.08003536002016114), (8, 0

[(10, -0.606923097499114), (5, 0.20930483733181554), (11, -0.13696806374323545), (9, 0.09854375724251244), (12, -0.09537679023557821), (0, 0.07997212095714215), (8, 0.04430487774676467), (13, 0.04214311637771389), (3, -0.03527703699315177), (6, 0.03374313829610446)]
[(10, -0.6338201435229271), (4, 0.1873062370359755), (5, -0.1790137388783914), (11, -0.1385341947502492), (9, 0.09722873740929641), (12, -0.09397794213780156), (7, -0.05826412470013338), (0, -0.05130803328224237), (8, 0.03772179118326752), (1, 0.03269633335982334)]
[(10, -0.5941411312257019), (4, 0.19363109497385536), (5, -0.1819113540099224), (3, 0.13112248354411946), (11, 0.13035616290127694), (9, 0.09450943550642339), (1, -0.07360285970944848), (8, 0.05019247806964756), (13, 0.034096318520969095), (12, 0.031155107076743706)]
[(10, -0.6123898166119495), (4, -0.15137775985543223), (11, -0.13989530680015047), (12, -0.08772716121822968), (9, -0.08482457134508721), (1, 0.07741138887252788), (0, 0.07688406955171513), (6, 0.071

[(10, -0.6527399878657631), (4, -0.15334985729072295), (11, -0.13352170803602856), (9, 0.0993668675701617), (12, -0.08653274244798795), (0, 0.06918541921947208), (3, 0.05612847413320814), (8, 0.04333885717011039), (13, 0.03250201047445931), (7, -0.027557330910481126)]
[(10, -0.6053637517873156), (5, 0.19711287150708562), (4, 0.19703986553384523), (11, -0.12356725887447331), (9, 0.10060109486231172), (12, -0.09544850431555699), (0, 0.07708737756030089), (6, 0.07049512055857936), (8, 0.043854864489535184), (13, 0.04095565747443296)]
[(10, -0.6182229518392623), (5, 0.19900935025128025), (4, 0.1919806841234434), (11, -0.15555623457490217), (9, 0.10054614921850905), (12, -0.09189244239460743), (8, 0.045597263967711854), (13, 0.0342438636293512), (7, 0.029999625398364083), (2, -0.0229459335748454)]
[(10, -0.62895282611373), (5, 0.21101816940038362), (11, -0.1416841964949947), (12, 0.11899755390419967), (9, 0.10732516439559321), (13, 0.051109107464552314), (8, 0.050680296739473184), (1, 0.031

lime.submodular_pick.SubmodularPick

In [110]:
from collections import Counter
a = Counter()
for i in sp_obj.V:
    exp = sp_obj.explanations[i]
    a1 = Counter(dict(exp.local_exp[1]))
    a.update(a1)

In [111]:
ans_data = []
for key in a:
    ans_data1 = []
    ans_data1.append(feature_names[key])
    ans_data1.append(a[key])
    ans_data.append(ans_data1)
df = pd.DataFrame(ans_data, columns = ["Feature", "Contribution"])
sumdf = df['Contribution'].sum()
df['Contribution'] = df['Contribution']
df

,Feature,Contribution
0,Capital Gain,-23.792107
1,Marital Status,2.116016
2,Education-Num,-1.573597
3,Capital Loss,-6.469338
4,Sex,1.804058
5,Hours per week,-2.496092
6,Age,0.698024
7,Race,1.927533
8,Country,-0.117665
9,Workclass,-0.070857


#### Explanation for original model (Note: We only consider top 10)

In [112]:
df.iloc[(-np.abs(df['Contribution'].values)).argsort()]


,Feature,Contribution
0,Capital Gain,-23.792107
3,Capital Loss,-6.469338
5,Hours per week,-2.496092
1,Marital Status,2.116016
7,Race,1.927533
4,Sex,1.804058
2,Education-Num,-1.573597
6,Age,0.698024
12,Education,0.667795
11,Relationship,0.235550


In [113]:
from lime import submodular_pick
sp_obj = submodular_pick.SubmodularPick(explainer, train, predict_fne, sample_size=500, num_features=10, num_exps_desired=50)
type(sp_obj)

[(10, 0.6381710548969938), (4, -0.156212270927697), (11, -0.11621245296039799), (7, -0.1128519123324365), (12, -0.08932689471243756), (5, -0.07811718945278498), (0, -0.06523393887399884), (9, 0.037527836756478465), (13, 0.024029742618161286), (2, -0.020972620392572014)]
[(10, -0.6262807087573802), (11, -0.15447186667516072), (12, -0.0887473478895777), (7, -0.07953297267715156), (5, -0.07774952519078984), (6, -0.07330971727787554), (9, 0.03903953118069666), (13, 0.030639023507829703), (0, -0.02293480439689863), (8, 0.01394595260920844)]
[(10, -0.60622701442585), (11, -0.1378498875852266), (7, -0.08863070237792954), (12, -0.0862406713178634), (6, 0.07803685852013485), (5, -0.07750612561846992), (9, -0.035446733175692774), (13, 0.035338906768702015), (2, 0.026542558204306985), (8, 0.022574962760410044)]
[(10, -0.6117476321044457), (4, -0.1552532269262953), (11, -0.13692633323742165), (0, 0.08752593226059494), (7, -0.0845867603691956), (12, -0.08398962915639269), (1, 0.06516218719605761), 

[(10, -0.6123302545334505), (4, -0.15043191570856582), (7, -0.13329130292948843), (11, -0.114071453444828), (12, -0.09028514870021677), (5, -0.08146326837542069), (9, -0.05108107106875722), (13, 0.03735977294516757), (8, 0.019743046158974716), (1, 0.0188652136484884)]
[(10, -0.6226513933888985), (4, -0.14903292056606768), (7, -0.09432247932222781), (11, -0.09341094359716384), (12, -0.08772644103739867), (9, 0.04931173356386163), (5, -0.04494091001941492), (13, 0.0447303759969647), (6, -0.03503095097383748), (0, -0.02284779774501258)]
[(10, -0.5955946636843008), (4, 0.1997649088452114), (11, -0.12866069183516682), (12, -0.09956547692154132), (7, -0.09521321388135907), (9, -0.05530447577072088), (5, -0.049238095624130215), (13, 0.02438038965989121), (2, -0.024322985664102535), (0, -0.019880902448435142)]
[(10, -0.6227574848162698), (11, -0.13665518124759465), (7, -0.08843858367430579), (12, -0.08806366235059052), (0, 0.08237412310379108), (9, -0.05228659840330493), (5, -0.040273783855969

[(10, -0.6175021842545512), (4, -0.15519091910026464), (11, -0.14109162399932582), (7, 0.12558825079879235), (5, 0.10208889852847763), (12, 0.09546002749369294), (1, -0.05934187931176626), (9, 0.04812104058493722), (13, 0.02954040381411462), (0, -0.028945868787193213)]
[(10, -0.6262590451022413), (11, -0.11751430802081023), (12, -0.09368831920898016), (7, -0.0841190360811431), (0, 0.06412132357540495), (4, 0.05619798031874494), (9, -0.03869274106844882), (6, -0.02375665229983592), (3, -0.02239499182190116), (13, 0.020925126221563603)]
[(10, -0.6215750292705016), (11, -0.1396977825453155), (7, 0.12642219256093398), (5, 0.10046917862567295), (12, -0.09230133553666955), (9, 0.05286509273460527), (4, 0.05171983692292842), (13, 0.026195971859454048), (1, 0.02291352056536586), (8, 0.020397192916484664)]
[(10, -0.6207400917324569), (7, 0.12375255315629177), (11, -0.10708187827355592), (5, 0.09908784898543194), (0, -0.07149561303036005), (9, 0.04499735001468786), (6, 0.04423093680347097), (4, 

[(10, 0.6245527186609031), (11, -0.14014531473883574), (7, -0.09059154069762738), (12, -0.08638321431306072), (4, 0.05691308501149645), (9, -0.04433170119613684), (1, -0.041133401928130905), (5, -0.0411321865573314), (3, -0.03542541489007785), (6, -0.03104314135351546)]
[(10, -0.6002138637011584), (4, -0.15690550817920662), (11, -0.14159811273952577), (12, -0.09065572938353017), (7, -0.08576266152923571), (6, -0.07965772039262532), (9, -0.051315731293547884), (3, 0.038624636290501654), (13, 0.03418848904190818), (1, 0.018840448613306665)]
[(10, -0.6170942745027592), (4, 0.1982412508978434), (7, 0.1319518251912277), (11, -0.1304507581982894), (5, 0.10291026614965695), (12, -0.08977832212699344), (0, 0.07314420710934202), (9, 0.05329575225581573), (8, 0.03910336310746966), (3, -0.025702083457360254)]
[(10, -0.6285197512777105), (4, 0.18666722177624587), (11, -0.11785656825626761), (12, 0.09489495521264733), (7, -0.09292065467642735), (0, 0.09017937127315598), (5, -0.08583287685636631), (

[(10, -0.6147562883254394), (4, -0.1530167649647965), (11, -0.11331125372717091), (7, -0.1115129095256295), (5, -0.08935166965836094), (12, -0.08603156029163365), (6, -0.08588385865484989), (0, -0.05082017565823747), (9, 0.047765383370048306), (3, -0.02442890493834223)]
[(10, -0.6303888247519889), (4, -0.15475426758371644), (11, -0.1405640824241524), (12, 0.10438393138926286), (5, -0.04710333046398591), (9, -0.045595482295805796), (0, -0.029788000205411846), (13, 0.02807502204673037), (2, 0.021384723046617777), (3, -0.018138937657744197)]
[(10, -0.6303112611222953), (11, -0.13168686098123772), (7, -0.08626392928129831), (5, -0.08150723230234419), (12, -0.0786572696844723), (9, -0.045583433622490004), (1, -0.02085188919840425), (13, 0.019830610795271115), (0, -0.018951005649277465), (3, -0.018308690566794326)]
[(10, -0.609519152758016), (11, -0.1284415529559677), (5, -0.09044485337606445), (7, -0.08952030314346959), (9, 0.06042131825992382), (0, -0.05413065270197815), (13, 0.02558535969

...

[(10, -0.617926971121554), (4, -0.15850550550110867), (11, -0.1396446712745067), (7, 0.12571138618708005), (5, 0.1077268879430317), (12, -0.10014744483364572), (0, 0.08637508796016466), (9, 0.04437473603177142), (3, 0.04426902962590471), (13, 0.035149796644512746)]
[(10, -0.6124925557294697), (4, -0.15680396821616246), (11, -0.14615752438241625), (7, -0.1175519033925575), (5, 0.09947281502748821), (12, -0.09229722771024473), (0, -0.050214709696916396), (6, -0.03976545941070724), (9, 0.0379057214641428), (13, 0.027792582714353576)]
[(10, -0.6185033874595249), (4, -0.15908013330209772), (11, -0.1504890563819976), (7, 0.12958706627493233), (5, 0.09519441405431234), (0, 0.0786245047796998), (9, 0.049899541896622615), (13, 0.03184245617810329), (2, 0.02477126074924994), (1, 0.020376038415050837)]
[(10, -0.6160928661718518), (11, -0.11451716102160102), (12, 0.1075379093227941), (7, -0.0843077951578195), (6, -0.06686640526142182), (9, -0.05032282292088415), (1, 0.028607055602061244), (13, 0.0

[(10, -0.6061083367111422), (4, -0.15806258086888003), (11, -0.1461328948087448), (7, -0.12989206707387121), (5, -0.09024702437773006), (12, -0.08616096304404361), (0, -0.050008809451480674), (9, 0.04493755981509141), (13, 0.014909215111029345), (1, 0.014020194938925378)]
[(10, -0.61173523146436), (4, 0.1940374873231391), (11, -0.15131712481968107), (7, 0.12766219399202164), (5, 0.10322182720475033), (12, -0.1010785738344224), (1, -0.06741219694427572), (9, 0.047143618826985575), (13, 0.02965217673709807), (3, -0.02437415418972952)]
[(10, -0.6293639176814905), (4, 0.18471330300349392), (11, -0.11863826308122524), (12, -0.08966194805764986), (5, -0.08526812295429316), (7, -0.0770303163280023), (0, -0.05345802312416926), (9, -0.03952423184198659), (6, 0.021961077606887484), (13, 0.01900231338134458)]
[(10, -0.5824649716543773), (11, -0.11936474184837452), (7, -0.11821432228130965), (12, -0.10245285907701505), (5, -0.08900348522858212), (0, -0.05585424625013591), (9, 0.05226467254498519),

[(10, -0.6023500640625822), (11, -0.155457657279605), (12, -0.08912622742931739), (5, -0.07369608892746436), (0, -0.05283389359246944), (13, 0.039864388031703564), (9, 0.03888554828331457), (2, 0.02313079382720498), (1, 0.02212979067300925), (4, -0.0113403102248093)]
[(10, -0.6285627892769577), (7, -0.12974493382117988), (11, -0.11781588863943275), (12, -0.1061077558065779), (5, -0.08277998130279422), (0, -0.0570930551946651), (9, 0.04572586911122372), (6, 0.03399737131921528), (13, 0.03193225813614902), (2, 0.029046927079815345)]
[(10, -0.6239214958676758), (4, 0.19671954412274814), (11, -0.14372007446733848), (7, 0.12246997837806785), (5, 0.10490112493071495), (12, 0.10283097020880225), (6, 0.09303820989059279), (1, 0.07854814215500851), (9, 0.04725043481139122), (0, -0.025715266008681632)]
[(10, -0.5850113960539112), (11, -0.13600637359886733), (7, -0.12094391349921278), (5, -0.09045432533953247), (12, -0.09020032162849684), (0, -0.05974717539853993), (9, -0.054333215408583914), (13

[(10, -0.6038417756482374), (11, -0.13357193538639361), (7, 0.12384377658148396), (12, 0.10777928807595513), (5, 0.10375691340078252), (0, -0.05869087743385858), (6, -0.05049437764392483), (9, 0.04520412629249924), (13, 0.026505553737376182), (8, 0.017105120877169398)]
[(10, -0.6255593207269045), (11, -0.14865144536264951), (7, -0.09354860779359407), (12, -0.08700186655743561), (5, -0.07834075338071836), (0, 0.07121279021006681), (9, -0.042445631963082056), (1, -0.03761094589830343), (8, -0.02624676206163123), (13, 0.023870601688482162)]
[(10, -0.6113905103365892), (4, -0.15119250281166152), (11, -0.14384481251308517), (7, -0.1192707735126224), (12, -0.08509379105878327), (6, -0.08415739733467013), (5, -0.07926994380591124), (0, -0.05910359717044176), (9, 0.04498843654353747), (13, 0.03208645117275655)]
[(10, -0.6122372735053071), (4, 0.18177372751773732), (11, -0.12397349739967713), (5, -0.09461784806641356), (12, 0.08849492172557523), (7, -0.08270772833655846), (9, -0.042336886683858

lime.submodular_pick.SubmodularPick

In [114]:
from collections import Counter
a = Counter()
for i in sp_obj.V:
    exp = sp_obj.explanations[i]
    a1 = Counter(dict(exp.local_exp[1]))
    a.update(a1)

In [115]:
ans_data = []
for key in a:
    ans_data1 = []
    ans_data1.append(feature_names[key])
    ans_data1.append(a[key])
    ans_data.append(ans_data1)
#     print(feature_names[key] )
df = pd.DataFrame(ans_data, columns = ["Feature", "Contribution"])
sumdf = df['Contribution'].sum()
df['Contribution'] = df['Contribution']
df

,Feature,Contribution
0,Capital Gain,-23.543842
1,Capital Loss,-5.767617
2,Education-Num,-1.673827
3,Relationship,0.065351
4,Marital Status,0.028490
5,Hours per week,-1.541263
6,Occupation,-0.062209
7,Age,-0.242858
8,Sex,0.477145
9,Country,0.802061


#### Explanation for LimeOut's Ensemble model (Note: We only consider top 10)

In [116]:
df.iloc[(-np.abs(df['Contribution'].values)).argsort()]


,Feature,Contribution
0,Capital Gain,-23.543842
1,Capital Loss,-5.767617
2,Education-Num,-1.673827
5,Hours per week,-1.541263
9,Country,0.802061
12,Education,0.547427
8,Sex,0.477145
10,Workclass,0.426351
7,Age,-0.242858
3,Relationship,0.065351


**a array has the accuracies for original model (calculated over 30 times)
b array has the accuracies for Ensemble model (calculated over 30 times)
Applied statistical t-test to see if their mean is significantly different. And verify that improvement in accuracy for Ensemble is significant**

In [117]:
a = [
82.65008444649163,
81.42177184093352,
82.6040227237832,
84.86027228989661,
82.35233903163068,
82.42399426758112,
82.42399426758112,
82.71061521138294,
83.03818200429932,
82.22950148428703,
82.59801412631795,
83.46811342000204,
82.21926502200839,
82.39328488074521,
83.26338417442932,
83.35636419468754,
83.11070167357593,
84.50790726239829,
83.50990327038231
]

In [118]:
b = [
81.97451251343468,
81.97451251343468,
82.65008444649163,
82.47517657897431,
82.46494011669567,
84.1539563926707,
84.1539563926707,
83.28385709898659,
84.45081379875116,
82.43423072985976,
82.34210256935202,
82.38304841846657,
83.22243832531477,
81.9121711536493,
83.58071450506705,
83.38622172177296,
81.1761093198219,
82.40442192538,
81.92845079072623
]

In [119]:
c = [
84.18547520343928,
84.60003070781514,
84.79963150621833,
83.89804483570478,
84.53270549698024,
84.47128672330842,
84.47128672330842,
83.63189681646023,
82.9767632306275,
82.58777766403931,
82.73108813594023,
83.60118742962432,
83.7342614392466,
82.58777766403931,
83.69331559013206,
84.14371993039205,
84.14371993039205,
82.97251650545063,
82.77291570704745
]

In [120]:
import scipy

In [121]:
scipy.stats.ttest_rel(a, b, axis=0, nan_policy='propagate')

Ttest_relResult(statistic=0.5163613830939042, pvalue=0.6118875682721937)

In [122]:
def remove_exponent(value):
    decial = value.split('e')
    ret_val = format(((float(decial[0]))*(10**int(decial[1]))), '.8f')
    return ret_val

In [123]:
scipy.stats.ttest_rel(b,c, axis=0, nan_policy='propagate')

Ttest_relResult(statistic=-3.899338846121264, pvalue=0.0010509638832705236)

In [124]:
scipy.stats.ttest_rel(a,c, axis=0, nan_policy='propagate')

Ttest_relResult(statistic=-2.8839235500158766, pvalue=0.009882440624063427)